In [1]:
# Advent of Code 2023
# Day 22 Problem 1
import re
from collections import Counter
import functools

with open("aoc_22_input.txt") as f:
    A = f.read().strip().split("\n")

# c+p test case here
TEST = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9""".strip().split("\n")

for i, line in enumerate(TEST):
    print(f"{i}:\t{line}")

0:	1,0,1~1,2,1
1:	0,0,2~2,0,2
2:	0,2,3~2,2,3
3:	0,0,4~0,2,4
4:	2,0,5~2,2,5
5:	0,1,6~2,1,6
6:	1,1,8~1,1,9


In [2]:
""" 
Each brick is a single straight line of cubes

sort by Z value

fall down, maintaining relationship of "supporting" and "supported by"

1. make bricks
2. fall as far down
3. iterate on bricks to resolve relationships
4. iterate for problem solution query
"""

from collections import deque

class ThreeDimGrid: 
    def __init__(self, input, printErrors=False) -> None:
        bricks = [SandBrick(i, line) for i, line in enumerate(input)]
        
        # if printErrors:
        #     print("Input bricks:")
        #     for b in bricks:
        #         b.pprint()
        #     print()

        # we want O(1) poplefts so use deque
        self.z_sorted_bricks = deque(sorted([brick for brick in bricks if brick.getZs()[0] > 1], key= lambda b: (b.getZs(), b.getXs(), b.getYs())))
        self.settled = sorted([brick for brick in bricks if brick.getZs()[0] == 1], key= lambda b: (b.getZs(), b.getXs(), b.getYs()))

        if printErrors:
            print("Settled bricks:")
            for b in self.settled:
                b.pprint()
            print()

            print("Bricks falling...")  

        self.gravity(printErrors)
        self.establishSupports()

        if printErrors:
            print()
            print("Settled bricks with supports:")
            for b in self.settled:
                b.pprintsups()
            print()

        return
    
    def gravity(self, printErrors=False):
        while self.z_sorted_bricks:
            curr = self.z_sorted_bricks.popleft()
            if printErrors:
                curr.pprint()

            cXMin, cXMax = curr.getXs()
            cYMin, cYMax = curr.getYs()
            cZMin, cZMax = curr.getZs()
            deltaZ = cZMax - cZMin

            candZs = []
            for lower in reversed(self.settled):
                lowerZMin, lowerZMax = lower.getZs()
                if lowerZMin >= cZMin:
                    continue

                lowerXMin, lowerXMax = lower.getXs()
                lowerYMin, lowerYMax = lower.getYs()
                # check for xy intersection
                if len(range(max(cXMin, lowerXMin), min(cXMax, lowerXMax)+1)) and len(range(max(cYMin, lowerYMin), min(cYMax, lowerYMax)+1)):
                    if printErrors:
                        print(f"current brick {curr.A} ~ {curr.B} will lay on cand brick {lower.A} ~ {lower.B} on Z-level {lowerZMax + 1}")
                        print(f"\tX: {range(max(cXMin, lowerXMin), min(cXMax, lowerXMax)+1)}\tY: {range(max(cYMin, lowerYMin), min(cYMax, lowerYMax)+1)}")
                    candZs.append(lowerZMax)

            if not candZs:
                curr.setZs(1, 1 + deltaZ)
            else:
                winner = max(candZs) + 1
                curr.setZs(winner, winner + deltaZ)
            self.settled.append(curr)
            
            if printErrors:
                curr.pprint()
                print()

    def establishSupports(self):
        sortedByMinZ = sorted(self.settled, key=lambda brick: (brick.getZs()))

        for curr in sortedByMinZ:
            cXMin, cXMax = curr.getXs()
            cYMin, cYMax = curr.getYs()
            cZMin, cZMax = curr.getZs()
            cands = list(filter(lambda b: b.getZs()[1] == cZMin - 1, self.settled ))
            #print(curr.id, [cand.id for cand in cands])

            for cand in cands:
                lowerXMin, lowerXMax = cand.getXs()
                lowerYMin, lowerYMax = cand.getYs()
                if len(range(max(cXMin, lowerXMin), min(cXMax, lowerXMax)+1)) and len(range(max(cYMin, lowerYMin), min(cYMax, lowerYMax)+1)):
                    # print(f"{'ABCDEFGHIJKLMNOPQRSTUVWXYZ'[brick.id]} is supported by {'ABCDEFGHIJKLMNOPQRSTUVWXYZ'[cand.id]}")
                    curr.addSupportedBy(cand.id)
                    cand.addSupport(curr.id)
    
    def jenga(self, printErrors=False):
        self.unsafeToRemove = set()
        for brick in self.settled:
            if len(brick.supportedBy) == 1:
                self.unsafeToRemove.add(brick.supportedBy[0])
        S = len(self.settled)
        U = len(self.unsafeToRemove)
        if printErrors:
            print(f"There are {S} bricks and {U} cannot be removed, leaving {S-U} safe to remove")
        return S - U


class SandBrick:
    def __init__(self, id, line) -> None:
        self.id = id
        coords = line.split("~")
        tmp1 = [int(x) for x in coords[0].split(",")]
        tmp2 = [int(x) for x in coords[1].split(",")]
        if tmp1[0] <= tmp2[0] and tmp1[1] <= tmp2[1] and tmp1[2] <= tmp2[2]:
            self.A = tmp1
            self.B = tmp2
        else:
            self.A = tmp2
            self.B = tmp1
        self.supports = []
        self.supportedBy = []

    def getXs(self):
        return [self.A[0], self.B[0]]

    def getYs(self):
        return [self.A[1], self.B[1]]

    def getZs(self):
        return [self.A[2], self.B[2]]
    
    def setZs(self, lo, hi):
        self.A[2] = lo
        self.B[2] = hi
    
    def pprint(self):
        print(f"[{self.id}] {self.A} ~ {self.B}")

    def pprintsups(self):
        print(f"[{self.id}] {self.A} ~ {self.B}")
        print(f"\tSupports: {self.supports}")
        print(f"\tSupported By: {self.supportedBy}")

    def addSupport(self, otherBrick):
        self.supports.append(otherBrick)
    
    def addSupportedBy(self, otherBrick):
        self.supportedBy.append(otherBrick)

def p1(input, printErrors=False):
    if printErrors:
        print(f"Initially we have {len(input)} bricks")
    grid = ThreeDimGrid(input, printErrors)
    return grid.jenga()

In [3]:
# expected output: 5
p1(TEST)

5

In [4]:
p1(A)
# 493 too high

454

In [5]:
# had a really hard time finding a code error so looked at some reddit test cases

TESTB = """0,0,1~0,1,1
1,1,1~1,1,1
0,0,2~0,0,2
0,1,2~1,1,2""".strip().split("\n")

TESTC = """0,0,1~1,0,1
0,1,1~0,1,2
0,0,5~0,0,5
0,0,4~0,1,4""".strip().split("\n")

TESTD = """0,0,1~0,0,2
1,0,1~2,0,1
1,0,2~1,0,2
0,0,3~1,0,3""".strip().split("\n")

TESTE = """5,1,1~1,1,1
1,5,2~1,1,2""".strip().split("\n")

TESTF = """0,0,1~0,0,1
1,1,1~1,1,1
0,0,2~0,1,2
0,1,3~1,1,3""".strip().split("\n")

TESTG = """0,0,2~0,0,4
1,0,3~2,0,3
1,0,4~1,0,5
0,0,6~1,0,6""".strip().split("\n")


In [6]:
# expected output: 3
p1(TESTB)

3

In [7]:
# expected output: 2
p1(TESTC)

2

In [8]:
# expected value: 3
p1(TESTD)

3

In [9]:
# expected value: 1
p1(TESTE)

1

In [10]:
# expected value: 2
p1(TESTF)

2

In [11]:
# expected value: 3
p1(TESTG)

3